<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

### pandas 라이브러리와 탐색적 데이터 분석 과정 익히기

> 다양한 데이터 분석 케이스를 통해 데이터 분석과 pandas 라이브러리 활용에 대해 익히보기로 합니다.

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보

### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- 평균 거래액은 얼마일까?
- 일별, 주별, 월별 판매 트렌드는?
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- **평균 배송 시간은?**
</div>

### 1. 평균 배송 시간 분석

In [ ]:
import pandas as pd
PATH = "00_data/"

In [ ]:
products = pd.read_csv(PATH + "olist_products_dataset.csv", encoding='utf-8-sig')
customers = pd.read_csv(PATH + "olist_customers_dataset.csv", encoding='utf-8-sig')
geolocation = pd.read_csv(PATH + "olist_geolocation_dataset.csv", encoding='utf-8-sig')
order_items = pd.read_csv(PATH + "olist_order_items_dataset.csv", encoding='utf-8-sig')
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
reviews = pd.read_csv(PATH + "olist_order_reviews_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
sellers = pd.read_csv(PATH + "olist_sellers_dataset.csv", encoding='utf-8-sig')
category_name = pd.read_csv(PATH + "product_category_name_translation.csv", encoding='utf-8-sig')

In [ ]:
orders.head()

#### 결측치 갯수 확인하기

In [ ]:
orders.info()

In [ ]:
orders.isnull().sum()

#### 결측치가 있는 행 삭제하기

In [ ]:
orders = orders.dropna()
orders.isnull().sum()

In [ ]:
orders.info()

#### 배송 시간을 계산해서 새로운 컬럼으로 추가
- 시간 차는 timedelta64 타입으로 표시됨

In [ ]:
orders['delivery_time'] = pd.to_datetime(orders['order_delivered_customer_date']) - pd.to_datetime(orders['order_purchase_timestamp'])

In [ ]:
orders.info()

### 2. 통계 활용
- 데이터가 주어졌을 때, 데이터 이해를 위해 가벼운 통계 기법을 사용 
  - 대표적인 예가 평균을 구하는 것임, 이외에 표준편차(std), 최소값(min), 사분위수(25%, 50%, 75%), 최대값(max)

- 데이터에 따라 보다 알맞은 대표값을 구하기 위해, 평균 이외에 다양한 기법이 존재함
  - 가중 평균
    - 데이터값($x_i$) X 가중치($w_i$) 의 총 합을 다시 가중치($w_i$)의 총합으로 나눈 것
    - 예: 여러 기기로부터 가져온 데이터 중, 특정 기기는 신뢰도가 떨어질 경우, 해당 기기로부터 나온 데이터에는 가중치를 낮게 줌
  - 중간값 (가중중간값도 가능)
    - 데이터를 정렬한 후 중간에 위치한 값을 취함
    - 평균은 특잇값(outlier)에 큰 영향을 받으므로, 특잇값에 큰 영향을 받지 않도록 중간값을 활용할 수 있음
  - 절사평균
    - 데이터를 정렬한 후, 양끝에서 일정 개수의 값들을 빼고, 남은 데이터를 기반으로 평균을 계산
    - 즉, 특잇값을 평균을 구할 때 제외하는 것임

In [ ]:
orders['delivery_time'].describe()

#### 2.1. boxplot 그래프로 특잇값의 정도 확인해보기
- 아래 예시는 **배송 시간(`delivery_time`)** 열을 **일 단위**로 변환한 뒤, Box Plot으로 시각화하여 y축 값이 보다 직관적으로 표시되도록 하는 코드
- Box Plot의 특잇값 구분 로직
  - Box Plot에서 특잇값을 판단하는 대표적인 규칙은 다음과 같습니다:
    - **Q1, Q3 계산**  
      - Q1: 하위 25% 지점 (1사분위수)  
      - Q3: 상위 25% 지점 (3사분위수)
    - **IQR(Interquartile Range) 계산**  
      - IQR = Q3 - Q1
    - **이상치(Outliers) 판정 범위**  
      - **하한값**: Q1 - (1.5 × IQR)  
      - **상한값**: Q3 + (1.5 × IQR)
    - **이 범위를 벗어난 데이터**는 특잇값(Outliers)으로 표시됨.
  - 데이터가 크게 편중되어 있으면(스큐가 심하면), 많은 관측치가 상한값을 초과하여 특잇값으로 분류될 수 있습니다.

In [ ]:
import pandas as pd
import plotly.express as px


# (2) 일(day) 단위로 변환: total_seconds()를 이용해 초 단위로 바꾼 뒤, 86400(하루 초 수)로 나눔
orders['delivery_time_days'] = orders['delivery_time'].dt.total_seconds() / 86400

# (3) Box Plot 생성: y축에 delivery_time_days 사용
fig = px.box(
    orders,
    y='delivery_time_days',
    title='Delivery Time Box Plot (in Days)',
    labels={'delivery_time_days': 'Delivery Time (Days)'}  # 축 레이블
)

# (4) 그래프 표시
fig.show()

#### 2.2. 상위에 있는 특잇값이 문제이므로, 0 ~ 95%에 해당하는 데이터만 사용해서 평균을 계산하기로 함
- pandas에서는 quantile() 함수를 제공
- 입력은 0 ~ 1 사이의 값으로, .95 는 95%에 해당하는 값을 의미

In [ ]:
delivery_time_q95 = orders['delivery_time'].quantile(.95)

In [ ]:
delivery_time_q95

In [ ]:
delivery_time_q90 = orders['delivery_time'].quantile(.90)
delivery_time_q90

In [ ]:
orders = orders[orders['delivery_time'] < delivery_time_q95]

In [ ]:
orders.info()

### 3. 월별 평균 배송 시간 분석
- orders_date['delivery_time'].dt.total_seconds() : 배송 시간을 초로 변환
  - float 타입으로 변환해서, mean() 평균 계산

In [ ]:
orders_date = orders[['order_purchase_timestamp', 'delivery_time']].copy()
# order_purchase_timestamp 의 날짜 데이터를 기반으로 월별 계산을 해야 하므로 datetime 타입으로 변환
orders_date['order_purchase_timestamp'] = pd.to_datetime(orders_date['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
# delivery_time이 timedelta64 타입인데, 이를 float 타입으로 변환
orders_date['delivery_time'] = orders_date['delivery_time'].dt.total_seconds()
orders_date = orders_date.set_index('order_purchase_timestamp')

In [ ]:
orders_date.info()

In [ ]:
orders_date = orders_date.groupby(pd.Grouper(freq='ME')).mean() # key 는 기본이 index 임
orders_date.head()

### 일 단위로 배송 시간 변환
- 1일은 86400초

In [ ]:
orders_date['delivery_time'] = orders_date['delivery_time'] / 86400

In [ ]:
orders_date.head()

In [ ]:
import plotly.express as px

# Plotly Express의 px.bar()를 사용하여 막대 그래프 생성
fig = px.bar(
    x=orders_date.index,             # x축: 주문 구매 날짜 (DataFrame의 인덱스)
    y=orders_date['delivery_time'],  # y축: 배송 시간 값
    title="Delivery Time by Order Purchase Timestamp",  # 그래프 제목
    labels={
        'x': 'Order Purchase Timestamp',  # x축 레이블
        'y': 'Delivery Time'                # y축 레이블
    }
)

# 긴 날짜 레이블이 겹치지 않도록 x축 라벨 각도 조정 (선택 사항)
fig.update_layout(xaxis_tickangle=-45)

# 그래프 출력
fig.show()


In [ ]:
orders_date = orders_date[orders_date.index > "2017-01-01"]

In [ ]:
import plotly.express as px

# orders_date DataFrame의 인덱스: order_purchase_timestamp, 컬럼: delivery_time
fig = px.bar(
    x=orders_date.index,             # x축: 주문 구매 날짜 (인덱스)
    y=orders_date['delivery_time'],  # y축: 배송 시간
    title="Delivery Time by Order Purchase Timestamp",  # 그래프 제목
    labels={
        'x': 'Order Purchase Timestamp',  # x축 레이블
        'y': 'Delivery Time'                # y축 레이블
    }
)

# 제공된 레이아웃 설정을 반영: x축 tick label, tick0, dtick, tickfont 등
fig.update_layout(
    xaxis=dict(
        showticklabels=True,       # x축 tick 라벨 표시 여부
        tick0="2017-01-31",        # 첫 번째 tick 라벨의 시작값
        dtick="M1",                # 월 단위 간격 (M1: 1개월 간격)
        tickfont=dict(size=7)      # tick 라벨의 글꼴 크기
    )
)

fig.show()


### plotly 로 세부적인 부분까지 수정해보기

In [ ]:
colors = ['#1B80BF'] * len(orders_date.index)
for index in range(15, len(orders_date.index)):
    colors[index] = '#BF2C47'

In [ ]:
import plotly.graph_objects as go  # Plotly의 그래프 객체 모듈을 임포트합니다.

# 새로운 Figure 객체 생성
fig = go.Figure()

# 막대 그래프(Bar Chart) trace 추가
fig.add_trace(
    go.Bar(
        x=orders_date.index,               # x축: orders_date DataFrame의 인덱스 (날짜)
        y=orders_date['delivery_time'],      # y축: 각 날짜별 평균 배송 시간
        text=orders_date['delivery_time'],   # 각 막대 위에 표시할 텍스트 (배송 시간)
        textposition='auto',                 # 텍스트 위치를 자동으로 설정
        texttemplate='%{text:.2f} days',      # 텍스트 포맷: 소수점 둘째 자리까지 표시 후 " days" 추가
        marker_color=colors                  # 막대의 색상을 colors 변수에 설정된 값으로 지정
    )
)

# 그래프의 레이아웃(제목, 축, 템플릿 등) 업데이트
fig.update_layout(
    {
        "title": {
            "text": "<b>Average Delivery Time per Month in Brazilian Olist E-Commerce company</b>",  # 그래프 제목 (HTML 태그로 굵게 표시)
            "x": 0.5,    # 제목을 수평 중앙(0.5) 위치에 배치
            "y": 0.9,    # 제목의 수직 위치를 0.9로 설정 (그래프 상단에 가깝게)
            "font": {
                "size": 15  # 제목의 글꼴 크기를 15로 지정
            }
        },
        "xaxis": {
            "title": "from Jan. 2017 to Aug. 2018",  # x축 제목 (날짜 범위)
            "showticklabels": True,                  # x축 눈금 라벨을 표시
            "tick0": "2017-01-31",                   # x축 시작 눈금을 2017-01-31로 설정
            "dtick": "M1",                           # x축 눈금 간격을 1개월 단위로 설정
            "tickfont": {
                "size": 7  # x축 눈금 라벨의 글꼴 크기를 7로 지정
            }
        },
        "yaxis": {
            "title": "Delivery Time (days)"  # y축 제목 (배송 시간 단위: days)
        },
        "template": 'plotly_white'  # 그래프 전체의 템플릿을 'plotly_white' (흰색 배경)로 설정
    }
)

# 특정 데이터 포인트에 주석(Annotation) 추가
fig.add_annotation(
    x="2018-04-30",                        # 주석의 x좌표: 2018-04-30 (날짜)
    y=10.4,                                # 주석의 y좌표: 10.4 (배송 시간)
    text="<b>Improved logistics</b>",      # 주석에 표시할 텍스트 (굵게 표시)
    showarrow=True,                        # 주석에 화살표를 표시하도록 설정
    font=dict(
        size=10,                           # 주석 텍스트의 글꼴 크기를 10으로 지정
        color="#ffffff"                    # 주석 텍스트의 색상을 흰색(#ffffff)으로 지정
    ),
    align="center",                        # 주석 텍스트를 중앙 정렬
    arrowhead=2,                           # 화살표 머리의 스타일을 2번으로 설정
    arrowsize=1,                           # 화살표의 크기를 1로 설정
    arrowwidth=2,                          # 화살표 선의 두께를 2로 설정
    arrowcolor="#77BDD9",                  # 화살표의 색상을 지정
    ax=40,                                 # 화살표 꼬리의 x축 오프셋(픽셀 단위)
    ay=-30,                                # 화살표 꼬리의 y축 오프셋(픽셀 단위)
    bordercolor="#77BDD9",                  # 주석 상자의 테두리 색상을 지정
    borderwidth=2,                         # 주석 상자의 테두리 두께를 2로 설정
    borderpad=4,                           # 주석 상자 내부의 패딩(여백)을 4로 설정
    bgcolor="#F22E62",                     # 주석 상자의 배경색을 지정
    opacity=0.8                            # 주석 상자의 불투명도를 0.8로 설정 (약간 투명)
)

# 최종 그래프를 화면에 표시
fig.show()

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>